In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.genmod.generalized_linear_model import GLM
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

data = pd.read_csv("./data/stdHTRU_2.csv")
train, test = train_test_split(data, test_size = 0.2)



NameError: name 'train_test_split' is not defined